<a href="https://colab.research.google.com/github/Albina1310/Albina1310/blob/main/SQL_%D0%B2_%D0%AF%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ базы данных книг

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

**Бизнес-цель** - сформулировать ценностное предложение для нового продукта.

**Наша цель** — проанализировать базу данных.

В базе — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Задачи, необходимые решить:**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Решим поставленные задачи

###  Задача 1 - посчитаем сколько книг вышло после 1 января 2000 года

In [2]:
query = '''
SELECT
count(book_id) AS count_book
FROM books
WHERE publication_date > '2000-01-01'
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count_book
0,819


После 1 января 2000 года из 1000 книг выпустилось 819.

### Задача 2 - для каждой книги посчитаем количество обзоров и среднюю оценку




In [3]:
query = '''
SELECT
b.book_id,
b.title,
round(avg(r.rating), 2) AS rating_avg,
count(DISTINCT rv.review_id) AS review_count
FROM books AS b
LEFT JOIN ratings AS r
ON b.book_id = r.book_id
LEFT JOIN reviews AS rv
ON b.book_id = rv.book_id
GROUP BY 1,2
ORDER BY 4 DESC, 3 DESC;
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,rating_avg,review_count
0,948,Twilight (Twilight #1),3.66,7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.41,6
2,299,Harry Potter and the Chamber of Secrets (Harry...,4.29,6
3,656,The Book Thief,4.26,6
4,734,The Glass Castle,4.21,6
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,4.00,0
996,387,Leonardo's Notebooks,4.00,0
997,221,Essential Tales and Poems,4.00,0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,0


Топ-5 книг популярных по количеству обзоров:
- Twilight (Twilight #1)
- The Hobbit or There and Back Again
- The Catcher in the Rye
- Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)
- Harry Potter and the Chamber of Secrets (Harry Potter  #2)

При этом средние оценки пользователей не самые высокие.

### Задача 3 - определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [4]:
query = '''
SELECT
p.publisher,
count(b.book_id) AS count_book
FROM books AS b
JOIN publishers AS p
ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY 2 DESC
limit 1
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_book
0,Penguin Books,42


Издательство Penguin Books выпустило наибольшее число книг - 42.

### Задача 4 - определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [5]:
query = '''
WITH count_rating AS (
SELECT
r.book_id,
a.author,
count(r.rating_id) AS count_rating,
avg(rating) AS avg_rating
FROM ratings AS r
LEFT JOIN books AS b
ON b.book_id = r.book_id
LEFT JOIN authors AS a
ON b.author_id = a.author_id
GROUP BY 1,2
HAVING count(r.rating_id) >= 50
)

SELECT
author,
avg(avg_rating) AS avg_rating_book
FROM count_rating
GROUP BY author
ORDER BY 2 DESC
LIMIT 1;
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating_book
0,J.K. Rowling/Mary GrandPré,4.283844


Среди книг получивших более 50 оценок, самым популярным автором с высокой средней оценкой является J.K. Rowling/Mary GrandPré.

### Задача 5 - посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [6]:
query = '''
WITH username AS(
SELECT
username,
count(rating_id) AS count_rating
FROM ratings
GROUP BY username
HAVING count(rating_id) > 48
),

review_count AS(
SELECT
u.username,
count(r.review_id) AS count_review
FROM username AS u
LEFT JOIN reviews AS r
ON u.username = r.username
GROUP BY u.username
)

SELECT
avg(count_review) AS avg_review
FROM review_count
'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review
0,24.0


Среди пользователей поставивших более 48 оценок среднее количество обзоров от них = 24.

## Вывод

**Выводы:** мы рассмотрели Базу Данных и вывели:
- нашли 819 книг выпустившихся после 1 января 2000 года
- топ-5 популярных книг по обзорам:
1. Twilight (Twilight #1)
2. The Hobbit or There and Back Again
3. The Catcher in the Rye
4. Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)
5. Harry Potter and the Chamber of Secrets (Harry Potter  #2)
- нашли самое популярное издательство, выпустившее наибольшее число книг:  Penguin Books
- нашли автора с самой высокой средней оценкой книг (среди авторов с числом оценок более 50): J.K. Rowling/Mary GrandPré
- нашли среднее количество обзоров от пользователей, поставивших более 48 оценок: 24 обзора.

Рекомендуется компании ориентироваться на популярные издательства, книги и авторов для формирования ценностного предложения для нового продукта.